In [6]:
from key import *
from langchain.chains import AnalyzeDocumentChain
from langchain.chat_models import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain
from key import *
# from pdfminer.high_level import extract_text
import fitz  # PyMuPDF
from pdfminer.high_level import extract_text

llm = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key=openai_key_private)

# qa_chain = load_qa_chain(llm, chain_type="map_reduce")

# qa_document_chain = AnalyzeDocumentChain(combine_docs_chain=qa_chain)

with open("datos/Factura1.pdf", "rb") as f:
    pdf_data = f.read()

# doc = fitz.open("pdf", pdf_data)

# Extrae el texto del PDF utilizando pdfminer.six
pdf_path = "datos/factura1.pdf"
text_content = extract_text(pdf_path)

# Mueve la creación y configuración de la cadena dentro del bucle
qa_chain = load_qa_chain(llm, chain_type="map_reduce")
qa_document_chain = AnalyzeDocumentChain(combine_docs_chain=qa_chain)
    
# texto_pdf = extract_text("datos/factura1.pdf")

response = qa_document_chain.run(
    input_document=text_content,
    # question="te voy a pasar una factura de la luz y quiero que me des los siguientes datos: CUPS, fecha de la factura, importe total, días de facturación (si no se especifica, encuentra la manera de calcularlo en base a fechas de uso del contador), energía reactiva, impuesto eléctrico, otros costes, descuento e IVA. En el caso de no haber algún dato escribe un 0.",
    question = """He extraído el texto de una factura de luz en formato PDF y necesito obtener información específica en formato JSON. Por favor, proporciona los siguientes datos de la factura en un formato clave-valor. Si algún dato no está claro o falta, solicita clarificaciones o indica un 0:

{
  "CUPS": "[Código Universal del Punto de Suministro]",
  "Fecha de la factura": "[Fecha específica]",
  "Importe total": "[Monto en euros]",
  "Días de facturación": "[Número de días o método de cálculo basado en fechas del contador]",
  "Energía reactiva": "[Cantidad en kWh o 0 si no aplica]",
  "Impuesto eléctrico": "[Monto en euros o 0 si no aplica]",
  "Alquiler contador": "[(también puede aparecer como equipos de medición, de medida, etc.) Monto en euros o 0 si no aplica]",
  "Descuento": "[Monto en euros o 0 si no aplica]",
  "IVA": "[Monto en euros o 0 si no aplica]"
}

Si algún dato no está disponible en la factura, reemplaza el valor con un 0. Una vez proporcionados estos datos, se espera que el usuario los valide para confirmar su exactitud."""
)

print(response)

{
  "CUPS": "ES0031405427789030NV0F",
  "Fecha de la factura": "12/09/2023",
  "Importe total": "42,64 €",
  "Días de facturación": "0",
  "Energía reactiva": "0",
  "Impuesto eléctrico": "3,78 €",
  "Alquiler contador": "0",
  "Descuento": "0",
  "IVA": "2,23 €"
}


In [7]:
import json
try:
    datos_factura = json.loads(response)
    print(datos_factura)
except json.JSONDecodeError as e:
    print(f"Error al decodificar JSON: {e}")

{'CUPS': 'ES0031405427789030NV0F', 'Fecha de la factura': '12/09/2023', 'Importe total': '42,64 €', 'Días de facturación': '0', 'Energía reactiva': '0', 'Impuesto eléctrico': '3,78 €', 'Alquiler contador': '0', 'Descuento': '0', 'IVA': '2,23 €'}


In [12]:
import requests_cache
from key import *
from langchain.chains import AnalyzeDocumentChain
from langchain.chat_models import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain
from key import *
import pdfplumber
# from pdfminer.high_level import extract_text
import fitz  # PyMuPDF

llm2 = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key=openai_key_private)

# Extrae el texto del PDF utilizando pdfminer.six
pdf_path = "datos/factura2.pdf"
text_content = extract_text(pdf_path)

# Cierra el documento después de extraer el texto
# doc.close()

# Limpiar la caché antes de realizar una nueva llamada
requests_cache.clear()

# Mueve la creación y configuración de la cadena dentro del bucle
qa_chain = load_qa_chain(llm2, chain_type="map_reduce")
qa_document_chain = AnalyzeDocumentChain(combine_docs_chain=qa_chain)
    
# texto_pdf = extract_text("datos/factura1.pdf")

response = qa_document_chain.run(
    input_document=text_content,
    # question="te voy a pasar una factura de la luz y quiero que me des los siguientes datos: CUPS, fecha de la factura, importe total, días de facturación (si no se especifica, encuentra la manera de calcularlo en base a fechas de uso del contador), energía reactiva, impuesto eléctrico, otros costes, descuento e IVA. En el caso de no haber algún dato escribe un 0.",
    question = """
        Extrae los siguientes datos de una factura de electricidad en formato clave-valor JSON:
        CUPS: Identificador único sin espacios.
        Fecha de la factura: En formato 'aaaa-mm-dd'.
        Importe total: Cifra total de la factura.
        Días de facturación: Número total de días cubiertos por la factura.
        Energía reactiva: Cantidad en kWh, si está disponible.
        Impuesto electricidad: Si es una factura de Endesa, extraer el importe del impuesto eléctrico.
        Otros costes: Cualquier otro costo mencionado.
        Alquiler equipo o contador: Importe asociado al alquiler del equipo o contador.
        Descuento: Cualquier descuento aplicado en la factura.
        P1 a P6 energía: Consumo en kW para los periodos Punta(P1), Llano(P2), Valle(P3), y otros si están disponibles, en Endesa figura en la segunda Pagina, en Información del consumo eléctrico.
        IVA: Porcentaje del IVA aplicado.
        Si algún dato no está disponible, se debe colocar un 0.
    """
)    
print(response)

{
  "CUPS": "ES0021000007484908MS0F",
  "Fecha de la factura": "2023-08-08",
  "Importe total": 51.70,
  "Días de facturación": 31,
  "Energía reactiva": 0,
  "Impuesto electricidad": 0.16,
  "Otros costes": 9.92,
  "Alquiler equipo o contador": 0.37,
  "Descuento": 0,
  "P1 energía": 29,
  "P2 energía": 21,
  "P3 energía": 3,
  "P4 energía": 0,
  "P5 energía": 0,
  "P6 energía": 0,
  "IVA": 5
}

{
  "CUPS": "ES1234567890123456",
  "Fecha de la factura": "2022-01-15",
  "Importe total": 50.00,
  "Días de facturación": 30,
  "Energía reactiva": 0,
  "Impuesto electricidad": 2.62,
  "Otros costes": 15.00,
  "Alquiler equipo o contador": 1.39,
  "Descuento": 0,
  "P1 energía": 29,
  "P2 energía": 21,
  "P3 energía": 3,
  "P4 energía": 0,
  "P5 energía": 0,
  "P6 energía": 0,
  "IVA": 5
}


In [43]:
print(text_content)

DATOS DE LA FACTURA

Nº factura: P23CON031719302
Referencia: 500326931734
Fecha emisión factura: 22/12/2023
Periodo de facturación: del 10/11/2023 a 15/12/2023 (35 días)
Fecha de cargo: 29 de diciembre de 2023
....................................................................................................................................................................

Endesa Energía, S.A. Unipersonal.
CIF A81948077.
C/Ribera del Loira, nº 60 28042 - Madrid.

MATIAS EZEQUIEL IBARRA
PJ ANDALUSIA 4 SOB 3
08014 BARCELONA
BARCELONA

RESUMEN DE LA FACTURA Y DATOS DE PAGO

Potencia
Energía
Descuentos
Otros
Impuestos
...........................................................................................................................................

17,02 €
18,39 €
-0,37 €
2,28 €
2,06 €

Total

39,38 €

(Detalle de la factura en el reverso)

INFORMACIÓN DEL CONSUMO ELÉCTRICO

De 10/11/2023 a 15/12/2023 (35 días)

......................................................................

In [30]:
import pdfplumber

with pdfplumber.open("datos/factura_endesa.pdf") as temp:
    for page_number in range(len(temp.pages)):
        current_page = temp.pages[page_number]

        # Obtén las dimensiones de la página
        width, height = current_page.width, current_page.height

        # Define las coordenadas para la primera mitad
        box_half_1 = (0, 0, width / 2, height)

        # Define las coordenadas para la segunda mitad
        box_half_2 = (width / 2, 0, width, height)

        # Extrae texto de la primera mitad
        text_half_1 = current_page.crop(box_half_1).extract_text()

        # Extrae texto de la segunda mitad
        text_half_2 = current_page.crop(box_half_2).extract_text()

        print(f"Texto de la página {page_number + 1}, Primera Mitad:\n{text_half_1}\n")
        print(f"Texto de la página {page_number + 1}, Segunda Mitad:\n{text_half_2}\n")


Texto de la página 1, Primera Mitad:
Endesa Energía, S.A. Unipersonal.
CIF A81948077.
C/Ribera del Loira, nº 60 28042 - Madrid.
RESUMEN DE LA FACTURA Y DATOS DE PAGO
Potencia 17,02 €
Energía 18,39 €
Descuentos -0,37 €
Otros 2,28 €
Impuestos 2,06 €
...........................................................................................................................................
Total 39,38 €
(Detalle de la factura en el reverso)
INFORMACIÓN DEL CONSUMO ELÉCTRICO
De 10/11/2023 a 15/12/2023 (35 días)
.........................................................................................................................................
Consumo Total 128,113 kWh
En esta factura el consumo
ha salido a0.140657 €/kWh
MATIAS, ¿sabes que tu consumo de ene
crecido un 90%?
Entra en infoEnergía, tu herramienta de
asesoramiento energético gratuita, y
ahorra en tu factura.
Infórmate en endesa.com/infoenergia
,802
oiloF
,0
orbiL
,797.21
omoT
.dirdaM
ed
litnacreM
ortsigeR
le
ne
atircsnI
.lanos

In [37]:
from pdfminer.high_level import extract_text

PDF_read = extract_text("datos/factura_endesa.pdf")

print(PDF_read)

DATOS DE LA FACTURA

Nº factura: P23CON031719302
Referencia: 500326931734
Fecha emisión factura: 22/12/2023
Periodo de facturación: del 10/11/2023 a 15/12/2023 (35 días)
Fecha de cargo: 29 de diciembre de 2023
....................................................................................................................................................................

Endesa Energía, S.A. Unipersonal.
CIF A81948077.
C/Ribera del Loira, nº 60 28042 - Madrid.

MATIAS EZEQUIEL IBARRA
PJ ANDALUSIA 4 SOB 3
08014 BARCELONA
BARCELONA

RESUMEN DE LA FACTURA Y DATOS DE PAGO

Potencia
Energía
Descuentos
Otros
Impuestos
...........................................................................................................................................

17,02 €
18,39 €
-0,37 €
2,28 €
2,06 €

Total

39,38 €

(Detalle de la factura en el reverso)

INFORMACIÓN DEL CONSUMO ELÉCTRICO

De 10/11/2023 a 15/12/2023 (35 días)

......................................................................